In [8]:
from art.attacks.extraction import CopycatCNN
from art.estimators.classification import KerasClassifier
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense
from keras.models import load_model, Sequential
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf
from art.defences.postprocessor import ReverseSigmoid



tf.compat.v1.disable_eager_execution()

In [2]:
# Carga del modelo creado
use_model = load_model('./model')
load_x = np.load('x_val.npy')
load_y = np.loadtxt('y_val.txt', delimiter=',')
X_train, X_test, y_train, y_test_ = train_test_split(load_x, load_y, test_size=0.30, random_state=1)

2023-05-30 18:26:32.245468: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:375] MLIR V1 optimization pass is not enabled
2023-05-30 18:26:32.271036: W tensorflow/c/c_api.cc:304] Operation '{name:'AssignVariableOp_3' id:109 op device:{requested: '/device:CPU:0', assigned: ''} def:{{{node AssignVariableOp_3}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false, _device="/device:CPU:0"](total_1, Identity_3)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


In [3]:
# Creacion del clasificador
keras_classifier = KerasClassifier(use_model)
copyCat = CopycatCNN(batch_size_fit=16, batch_size_query=16, nb_epochs=5,nb_stolen=500,classifier=keras_classifier)

In [4]:
sequential_model = Sequential()
sequential_model.add(Convolution2D(32, (3, 3), strides=(1, 1), input_shape=(256, 256, 3), activation='relu'))
sequential_model.add(MaxPooling2D(pool_size=(2, 2)))
sequential_model.add(Convolution2D(64, (3, 3), strides=(1, 1), activation='relu'))
sequential_model.add(MaxPooling2D(pool_size=(2, 2)))
sequential_model.add(Flatten())
sequential_model.add(Dense(512, activation='softplus'))
sequential_model.add(Dense(26, activation='sigmoid'))
sequential_model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])

In [5]:
model_stolen = KerasClassifier(model=sequential_model)

In [6]:
stolen_classifier = copyCat.extract(
  x=X_train, 
  y=y_train, 
  thieved_classifier=model_stolen
)

/Users/oscarsaravia/Documents/Universidad/security-data-science/LAB8DSD/venv/lib/python3.11/site-packages/keras/src/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
2023-05-30 18:26:35.989535: W tensorflow/c/c_api.cc:304] Operation '{name:'dense_1/Sigmoid' id:150 op device:{requested: '', assigned: ''} def:{{{node dense_1/Sigmoid}} = Sigmoid[T=DT_FLOAT, _has_manual_control_dependencies=true](dense_1/BiasAdd)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-05-30 18:26:36.007147: W tensorflow/c/c_api.cc:304] Operation '{name:'count_3/Assign' id:426 op device:{requested: '', assigned: ''} def:{{{node count_3/Assign}} = AssignVariableOp[_has_manual_cont

Train on 500 samples


2023-05-30 18:26:40.416467: W tensorflow/c/c_api.cc:304] Operation '{name:'loss_1/mul' id:492 op device:{requested: '', assigned: ''} def:{{{node loss_1/mul}} = Mul[T=DT_FLOAT, _has_manual_control_dependencies=true](loss_1/mul/x, loss_1/dense_1_loss/value)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-05-30 18:26:40.438302: W tensorflow/c/c_api.cc:304] Operation '{name:'training/Adam/dense_2/bias/v/Assign' id:749 op device:{requested: '', assigned: ''} def:{{{node training/Adam/dense_2/bias/v/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](training/Adam/dense_2/bias/v, training/Adam/dense_2/bias/v/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Eith

Epoch 1/5
500/500 [==============================] - 19s 38ms/sample - loss: 584.4945 - accuracy: 0.7920
Epoch 2/5
500/500 [==============================] - 18s 35ms/sample - loss: 1.1877 - accuracy: 0.9000
Epoch 3/5
500/500 [==============================] - 18s 36ms/sample - loss: 0.4811 - accuracy: 0.9100
Epoch 4/5
500/500 [==============================] - 17s 33ms/sample - loss: 0.3604 - accuracy: 0.9160
Epoch 5/5
500/500 [==============================] - 17s 35ms/sample - loss: 0.2419 - accuracy: 0.9260


In [7]:
score1 = keras_classifier._model.evaluate(x=X_test, y=y_test_)
score2 = stolen_classifier._model.evaluate(x=X_test,y=y_test_)
print(score1)
print(score2)

/Users/oscarsaravia/Documents/Universidad/security-data-science/LAB8DSD/venv/lib/python3.11/site-packages/keras/src/engine/training_v1.py:2335: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates
2023-05-30 18:28:09.447145: W tensorflow/c/c_api.cc:304] Operation '{name:'loss/mul' id:226 op device:{requested: '', assigned: ''} def:{{{node loss/mul}} = Mul[T=DT_FLOAT, _has_manual_control_dependencies=true](loss/mul/x, loss/dense_1_loss/value)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-05-30 18:28:12.847926: W tensorflow/c/c_api.cc:304] Operation '{name:'loss_1/mul' id:492 op device:{requested: '', assigned: ''} def:{{{node loss_1/mul}} = Mul[T=DT_FLOAT, _has_manual_control_de

[32.65007080078125, 0.6616667]
[4.607252693176269, 0.73]


Discusión
<br>
* Desde nuestro punto de vista, el ataque se puede mejorar. Los resultados obtenidos se deben a la cantidad de epochs y el tamaño del modelo que se utilizó.

### AQUI INICIA LA DEFENSA (LAB08)


In [9]:
def getK_Model():
  sequential_model = Sequential()
  sequential_model.add(Convolution2D(32, (3, 3), strides=(1, 1), input_shape=(256, 256, 3), activation='relu'))
  sequential_model.add(MaxPooling2D(pool_size=(2, 2)))
  sequential_model.add(Convolution2D(64, (3, 3), strides=(1, 1), activation='relu'))
  sequential_model.add(MaxPooling2D(pool_size=(2, 2)))
  sequential_model.add(Flatten())
  sequential_model.add(Dense(512, activation='softplus'))
  sequential_model.add(Dense(26, activation='sigmoid'))
  sequential_model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])
  sequential_model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])
  return sequential_model

reverser_sigmoid = ReverseSigmoid(beta=1.0,gamma=0.2)
kClass = KerasClassifier(model=use_model,postprocessing_defences=reverser_sigmoid,)
kClass_kmodel = KerasClassifier(model=getK_Model())
copycat_cnn = CopycatCNN(batch_size_fit=16,batch_size_query=16,nb_epochs=5,nb_stolen=len(X_train),classifier=kClass)
stolen_classifier = copycat_cnn.extract(x=X_train, y=y_train, thieved_classifier=kClass_kmodel)

2023-05-30 18:39:57.125350: W tensorflow/c/c_api.cc:304] Operation '{name:'total_4/Assign' id:1019 op device:{requested: '', assigned: ''} def:{{{node total_4/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](total_4, total_4/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
/Users/oscarsaravia/Documents/Universidad/security-data-science/LAB8DSD/venv/lib/python3.11/site-packages/art/defences/postprocessor/reverse_sigmoid.py:75: RuntimeWarning: divide by zero encountered in log
  perturbation_r = self.beta * (sigmoid(-self.gamma * np.log((1.0 - preds_clipped) / preds_clipped)) - 0.5)


Train on 1400 samples


2023-05-30 18:40:09.024426: W tensorflow/c/c_api.cc:304] Operation '{name:'loss_3/mul' id:1166 op device:{requested: '', assigned: ''} def:{{{node loss_3/mul}} = Mul[T=DT_FLOAT, _has_manual_control_dependencies=true](loss_3/mul/x, loss_3/dense_3_loss/value)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-05-30 18:40:09.056448: W tensorflow/c/c_api.cc:304] Operation '{name:'training_2/Adam/conv2d_3/bias/m/Assign' id:1366 op device:{requested: '', assigned: ''} def:{{{node training_2/Adam/conv2d_3/bias/m/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](training_2/Adam/conv2d_3/bias/m, training_2/Adam/conv2d_3/bias/m/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in th

Epoch 1/5
1400/1400 [==============================] - 56s 40ms/sample - loss: 545.8157 - accuracy: 0.3229
Epoch 2/5
1400/1400 [==============================] - 48s 35ms/sample - loss: 1.6154 - accuracy: 0.3643
Epoch 3/5
1400/1400 [==============================] - 49s 35ms/sample - loss: 1.5093 - accuracy: 0.4221
Epoch 4/5
1400/1400 [==============================] - 49s 35ms/sample - loss: 0.9417 - accuracy: 0.6636
Epoch 5/5
1400/1400 [==============================] - 50s 36ms/sample - loss: 0.2609 - accuracy: 0.9214


In [11]:
score_original = kClass._model.evaluate(x=X_test,y=y_test_)
score_stolen = stolen_classifier._model.evaluate(x=X_test,y=y_test_)

print('Loss')
print('Original Loss: ',score_original[0])
print('Stolen Loss: ',score_stolen[0])
print('Accuracy')
print('Original Accuracy: ',score_original[1])
print('Stolen Accuracy: ',score_stolen[1])


Loss
Original Loss:  32.65007080078125
Stolen Loss:  6.25589682896932
Accuracy
Original Accuracy:  0.6616667
Stolen Accuracy:  0.0


#### Discusion
Despues de ejecutar el programa y ver los resultados, podemos observar que la defensa fue buena, esto debido a que en comparacion con el accuracy anterior, la precision nos indica que el modelo de defensa utilizado es efectivo.